In [1]:
from argparse import Namespace

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
import seaborn as sns
import os
import glob
import sklearn
import nilearn as nl
from sklearn.manifold import TSNE
import re
from nilearn.input_data import NiftiMasker
from sklearn.cluster import KMeans
import math 
from numpy import dot
from numpy.linalg import norm
import time

/usr/local/lib/python3.10/dist-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [3]:
participants = pd.read_csv('/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/train/train_labels/participants.tsv', sep="\t")  
age_gender = participants.loc[:, ["age", "sex"]]
duplicates = participants["participant_id"][participants["participant_id"].isin(participants["participant_id"][participants["participant_id"].duplicated()])]


In [4]:
def loadFileNames(path, type):
    return  glob.glob(os.path.join(path, type))


In [11]:
def laodFiles(files):
    i=0
    sub_info = pd.DataFrame(dtype=float)
    feat_set = np.load(files[0])
    for file in files:
        file_i = np.load(file)
        id = re.search("sub-(.*)_prep", file)
        id = round(int(id.group(1)), -6)
        age = participants["age"].loc[participants['participant_id'] == id]
        age = age.astype(float)
        if id not in duplicates.values:
            sub_info = sub_info.append([[id,age.values]], ignore_index=True)
            feat_set = np.concatenate((feat_set, file_i))
        i = i+1

        if i==1100:
            break
    
    sub_info.iloc[:, 1] = sub_info.iloc[:,1].astype(float)
    return feat_set, sub_info

T-SNE projection of the feature matrix with k-means age clusters

In [33]:
# sub_metadata = pd.read_csv('/media/dataanalyticlab/Drive2/MANSOOR/Dataset/openbhb_sub_metadata_k-means_cluster.csv')
# reduced_train_matrix = pd.read_csv('/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/Reduced_matrices_openbhb/openbhb_train_roi_dataset.csv', header=None, usecols=np.arange(1,1797))
reduced_train_matrix = np.load("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/Reduced_matrices_openbhb/CAT12_VBM_features_Quasi_Raw_features_PCA_openbhb_train_dataset.npy")
reduced_train_matrix = pd.DataFrame(reduced_train_matrix)


In [14]:
tsne = pd.DataFrame(np.load("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/Reduced_matrices_openbhb/reduced_tSNE_openbhb_train_dataset.npy"))

In [34]:
k = 3
# findAgeClusters(sub_metadata, k)
# clusters = findClusterIntervals(sub_metadata , k)
for i in range(50,500, 50):
    tsne = findTSNEcomp(reduced_train_matrix, sub_metadata, perplexity=i, k=3)
    print("Perplexity = ", i)
    fig = PlotTSNE(tsne, k)
    # plt.show()
    
    fig.savefig("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/EDA/tSNE_openbhb/PCA_1000_components/tsne_quasiraw_cat12_1000-pca_"+str(i)+"-perp_"+str(k)+"-clusters.png")
    plt.close(fig)


Perplexity =  50
Perplexity =  100
Perplexity =  150
Perplexity =  200
Perplexity =  250
Perplexity =  300
Perplexity =  350
Perplexity =  400
Perplexity =  450


In [24]:
# plot tsne projection of the tsne components derived already 
tsne_feat = pd.DataFrame()
tsne_feat["Cluster_ID"] = sub_metadata.loc[:, "Cluster_ID(" + str(k)+"-means)"]
tsne_feat["age-cluster"] = sub_metadata.loc[:, "age-interval(" + str(k)+"-means)"]
tsne_feat["Age"] = sub_metadata.iloc[:, 2]
tsne_feat["comp-1"] = tsne.iloc[:,0]
tsne_feat["comp-2"] = tsne.iloc[:,1]
fig = PlotTSNE(tsne_feat,4)
fig.savefig("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/EDA/tSNE_openbhb/PCA_1000_components/tsne_reduced_"+str(i)+"-perp_"+str(k)+"-clusters.png")
plt.close(fig)


In [92]:
def findAgeClusters(sub_info, k):
    kmeans = KMeans(k)
    ages = np.array(sub_info.iloc[:,1])
    ages = ages.reshape(ages.shape[0],-1)
    clusters = kmeans.fit_predict(ages)
    sub_info["Cluster_ID("+str(k)+ "-means)"] = clusters

In [104]:
def findClusterIntervals(sub_info, k):
    intervals = sub_info.groupby("Cluster_ID("+str(k)+ "-means)").agg(Min_Age=(1, 'min'),
    Max_Age=(1, 'max'),Mean_Age=(1, 'mean'), Frequency=(1,"count")).reset_index()
    intervals.iloc[:,:3] = round(intervals.iloc[:,:3], 2).astype(int)
    mapping = {0: str(intervals.iloc[0,1]) + " - " + str(intervals.iloc[0,2]), 
    1: str(intervals.iloc[1,1]) + " - " + str(intervals.iloc[1,2]),
    2: str(intervals.iloc[2,1]) + " - " + str(intervals.iloc[2,2]),
    3: str(intervals.iloc[3,1]) + " - " + str(intervals.iloc[3,2]),
    4: str(intervals.iloc[4,1]) + " - " + str(intervals.iloc[4,2]),
    5: str(intervals.iloc[5,1]) + " - " + str(intervals.iloc[5,2]),
    6: str(intervals.iloc[6,1]) + " - " + str(intervals.iloc[6,2]),
    7: str(intervals.iloc[7,1]) + " - " + str(intervals.iloc[7,2])
    }
    cluster = [mapping[i] for i in sub_info["Cluster_ID("+str(k)+ "-means)"]]
    sub_info["age-interval("+str(k)+ "-means)"] = cluster
    return intervals
    
    

In [22]:
def findTSNEcomp(features, sub_info, perplexity, k):
    tsne = TSNE(n_components=2, perplexity=perplexity)
    tsne = tsne.fit_transform(features)
    tsne_feat = pd.DataFrame()
    tsne_feat["Cluster_ID"] = sub_metadata.loc[:, "Cluster_ID(" + str(k)+"-means)"]
    tsne_feat["age-cluster"] = sub_metadata.loc[:, "age-interval(" + str(k)+"-means)"]
    tsne_feat["Age"] = sub_metadata.iloc[:, 2].astype(int)
    tsne_feat["comp-1"] = tsne[:,0]
    tsne_feat["comp-2"] = tsne[:,1]
    return tsne_feat
     

In [5]:
def PlotTSNE(tsne, k):
    tsne_sort = tsne.sort_values(by="Age")
    fig = plt.figure()
    sns.scatterplot(x="comp-1", y="comp-2", hue=tsne_sort.loc[:, "age-cluster"],
     color=["r", "g", "b", "y"], palette=sns.color_palette("hls", k), 
                data=tsne_sort.loc[:, ["comp-1", "comp-2"]])
    return fig


In [5]:
path = '/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/train/train_fsl/'
files = loadFileNames(path, "*xhemi_T1w.npy")
# features, subjects = laodFiles(files)
# feature_set = pd.DataFrame(features.reshape(features.shape[0],-1)).iloc[1:,:]
# subjects_subset1 = subjects
# tsne = TSNE(n_components=2, perplexity=7)
# tsne_subset1 = tsne.fit_transform(feature_set)
# findAgeClusters(5,subjects)
# clusters = findClusterIntervals(subjects)
# tsne = findTSNEcomp(feature_set, subjects)
# PlotTSNE(tsne, 5)